In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#### Constants

In [24]:
RANKS = "_?W23456789XJQKA"
SUITS = "_?W♣♦♥♠"

#### Helpers

In [ ]:
def card_str_to_tuple(card_str):
    rank_char, suit_char = card_str
    rank = RANKS.index(rank_char)
    suit = SUITS.index(suit_char)
    return (rank, suit)

def classify_poker_hand(card_tuples):
    ranks = [rank for rank, _ in card_tuples if rank > 2]
    rank_counts = {rank: ranks.count(rank) for rank in set(ranks)}
    wild_ranks = [rank for rank, _ in card_tuples if rank == 2]
    is_flush = len(set(suit for _, suit in card_tuples if suit > 1)) == 1
    sorted_ranks = sorted(ranks)
    # check for straight (including low-Ace straight)
    is_straight = False
    if len(sorted_ranks) >= 5:
        if sorted_ranks == list(range(sorted_ranks[0], sorted_ranks[0] + 5)):
            is_straight = True
        elif sorted_ranks[-4:] == [10, 11, 12, 13] and sorted_ranks[0] == 14:
            is_straight = True

    if is_straight and is_flush and sorted_ranks[-1] == 14:
        return "royal flush"
    elif is_straight and is_flush:
        return "straight flush"
    elif 4 in rank_counts.values():
        return "four of a kind"
    elif 3 in rank_counts.values() and 2 in rank_counts.values():
        return "full house"
    elif is_flush:
        return "flush"
    elif is_straight:
        return "straight"
    elif 3 in rank_counts.values():
        return "three of a kind"
    elif list(rank_counts.values()).count(2) == 2:
        return "two pair"
    elif 2 in rank_counts.values():
        return "one pair"
    elif len(ranks) > 0:
        return "high card"
    else:
        return "nothing"

def poker_hand_label_to_index(label):
    label_map = {
        "nothing": 0,
        "high card": 1,
        "one pair": 2,
        "two pair": 3,
        "three of a kind": 4,
        "straight": 5,
        "flush": 6,
        "full house": 7,
        "four of a kind": 8,
        "straight flush": 9,
        "royal flush": 10,
    }
    return label_map[label]

def reverse_poker_hand_index(index):
    index_map = {
        0: "nothing",
        1: "high card",
        2: "one pair",
        3: "two pair",
        4: "three of a kind",
        5: "straight",
        6: "flush",
        7: "full house",
        8: "four of a kind",
        9: "straight flush",
        10: "royal flush",
    }
    return index_map[index]

#### Encoders

In [ ]:
class CardEncoder(nn.Module):
    def __init__(self, ranks=len(RANKS), suits=len(SUITS), rank_dim=8, suit_dim=4, output_dim=16):
        super().__init__()
        self.rank_emb = nn.Embedding(ranks, rank_dim)
        self.suit_emb = nn.Embedding(suits, suit_dim)
        self.fc = nn.Linear(suit_dim + rank_dim, output_dim)

    def forward(self, card_tensor):
        rank_tensor = card_tensor[:, 0]
        suit_tensor = card_tensor[:, 1]
        rank_emb = self.rank_emb(rank_tensor)
        suit_emb = self.suit_emb(suit_tensor)
        combined = torch.cat((rank_emb, suit_emb), dim=-1)
        output = self.fc(combined)
        return output

class PokerHandClassifier(nn.Module):
    def __init__(self, input_dim=16, hidden_dim=32, num_classes=11):
        super().__init__()
        self.fc1 = nn.Linear(input_dim * 5, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, hand_tensor):
        batch_size = hand_tensor.size(0)
        hand_flat = hand_tensor.view(batch_size, -1)
        x = F.relu(self.fc1(hand_flat))
        output = self.fc2(x)
        return output

################################################################
################################################################

class PokerHandClassifier_WithAttention(nn.Module):
    def __init__(self, input_dim=16, hidden_dim=32, num_classes=11, num_heads=4):
        super().__init__()
        # self-attention layer lets cards "interact"
        self.attn = nn.MultiheadAttention(embed_dim=input_dim, num_heads=num_heads, batch_first=True)
        
        # optional feedforward layer after attention
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, hand_tensor):
        # hand_tensor: (batch, num_cards, input_dim)
        attn_output, _ = self.attn(hand_tensor, hand_tensor, hand_tensor)
        
        # permutation-invariant pooling (mean or sum)
        pooled = attn_output.mean(dim=1)  # (batch, input_dim)
        
        x = F.relu(self.fc1(pooled))
        output = self.fc2(x)
        return output

################################################################
################################################################

class PokerHandClassifier_SelfAttention(nn.Module):
    def __init__(self, input_dim=16, hidden_dim=32, num_classes=11, num_heads=4, num_layers=2):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.MultiheadAttention(embed_dim=input_dim, num_heads=num_heads, batch_first=True)
            for _ in range(num_layers)
        ])
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, hand_tensor):
        x = hand_tensor
        for attn in self.layers:
            attn_out, _ = attn(x, x, x)
            x = x + attn_out  # residual connection
        pooled = x.mean(dim=1)
        x = F.relu(self.fc1(pooled))
        return self.fc2(x)

################################################################
################################################################

class AttentionPooling(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.attn = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        # x: (batch, num_cards, input_dim)
        attn_scores = self.attn(x)                 # (batch, num_cards, 1)
        attn_weights = torch.softmax(attn_scores, dim=1)
        pooled = torch.sum(attn_weights * x, dim=1)  # weighted sum (batch, input_dim)
        return pooled

class PokerHandClassifier_AttentionPooling(nn.Module):
    def __init__(self, input_dim=16, hidden_dim=32, num_classes=11):
        super().__init__()
        self.pool = AttentionPooling(input_dim, hidden_dim)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, hand_tensor):
        pooled = self.pool(hand_tensor)
        x = F.relu(self.fc1(pooled))
        return self.fc2(x)

################################################################
################################################################

class SAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads=4):
        super().__init__()
        self.mha = nn.MultiheadAttention(embed_dim=dim_out, num_heads=num_heads, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(dim_out, dim_out * 2),
            nn.ReLU(),
            nn.Linear(dim_out * 2, dim_out),
        )
        self.ln1 = nn.LayerNorm(dim_out)
        self.ln2 = nn.LayerNorm(dim_out)
        self.proj = nn.Linear(dim_in, dim_out) if dim_in != dim_out else nn.Identity()

    def forward(self, x):
        # Project to match MHA input dimension if necessary
        x_proj = self.proj(x)
        attn_out, _ = self.mha(x_proj, x_proj, x_proj)
        x = self.ln1(x_proj + attn_out)
        ff_out = self.fc(x)
        return self.ln2(x + ff_out)

class PMA(nn.Module):
    def __init__(self, dim, num_heads=4, num_seeds=1):
        super().__init__()
        self.seed_vectors = nn.Parameter(torch.randn(num_seeds, dim))
        self.mha = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, batch_first=True)

    def forward(self, x):
        batch_size = x.size(0)
        seed = self.seed_vectors.unsqueeze(0).repeat(batch_size, 1, 1)  # (B, num_seeds, dim)
        out, _ = self.mha(seed, x, x)
        return out  # (B, num_seeds, dim)

class PokerHandClassifier_SetTransformer(nn.Module):
    def __init__(self, input_dim=16, dim_hidden=32, num_heads=4, num_classes=11):
        super().__init__()
        self.sab1 = SAB(input_dim, dim_hidden, num_heads)
        self.sab2 = SAB(dim_hidden, dim_hidden, num_heads)
        self.pma = PMA(dim_hidden, num_heads, num_seeds=1)
        self.fc = nn.Sequential(
            nn.Linear(dim_hidden, dim_hidden),
            nn.ReLU(),
            nn.Linear(dim_hidden, num_classes),
        )

    def forward(self, hand_tensor):
        # hand_tensor: (batch, num_cards, input_dim)
        x = self.sab1(hand_tensor)
        x = self.sab2(x)
        x = self.pma(x).squeeze(1)  # (batch, dim_hidden)
        return self.fc(x)

In [ ]:
# ---- convert card strings to tensor ----
def card_strings_to_tensor(card_strings):
    card_tuples = [card_str_to_tuple(cs) for cs in card_strings]
    card_tensor = torch.tensor(card_tuples, dtype=torch.long)
    return card_tensor

if __name__ == "__main__":
    # ---- training set ----
    card_strings = [
        ["A♥", "K♦", "Q♣", "J♠", "__"], # high card
        ["A♥", "K♥", "Q♥", "J♥", "X♥"], # royal flush
        ["9♠", "8♠", "7♠", "6♠", "5♠"], # straight flush
        ["3♦", "3♣", "3♥", "3♠", "9♥"], # four of a kind
        ["4♥", "4♠", "4♣", "9♦", "9♥"], # full house
        ["2♣", "5♣", "9♣", "J♣", "K♣"], # flush
        ["6♥", "5♠", "4♦", "3♣", "2♥"], # straight
        ["7♠", "7♥", "7♣", "2♦", "5♥"], # three of a kind
        ["8♦", "8♣", "4♥", "4♠", "K♥"], # two pair
        ["J♣", "J♠", "3♥", "6♦", "9♣"], # one pair
        ["5♥", "X♠", "7♣", "4♦", "2♥"], # high card
        ["5♥", "5♠", "__", "__", "__"], # one pair with 3 empty cards
        ["9♣", "9♦", "4♥", "4♠", "??"], # two pair and 1 ?? card
        ["__", "__", "__", "__", "__"], # empty hand
    ]

    card_tensors = []
    labels = []
    for hand in card_strings:
        card_tensor = card_strings_to_tensor(hand)
        card_tensors.append(card_tensor)
        card_tuples = [card_str_to_tuple(cs) for cs in hand]
        label = classify_poker_hand(card_tuples)
        labels.append(poker_hand_label_to_index(label))
    card_tensors = torch.stack(card_tensors)
    labels = torch.tensor(labels, dtype=torch.long)
    print("Card Tensors:\n", card_tensors)
    print("Labels:\n", labels)
    encoder = CardEncoder()
    # ---- choose classifier ----
    classifier = PokerHandClassifier()
    # ---- choose classifier ----
    encoded_cards = encoder(card_tensors.view(-1, 2))
    encoded_hands = encoded_cards.view(card_tensors.size(0), 5, -1)
    outputs = classifier(encoded_hands)
    print("Classifier Outputs:\n", outputs)


In [ ]:
# ---- train loop ----
def train_loop(model, data_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for batch_cards, batch_labels in data_loader:
            optimizer.zero_grad()
            encoded_cards = encoder(batch_cards.view(-1, 2))
            encoded_hands = encoded_cards.view(batch_cards.size(0), 5, -1)
            outputs = model(encoded_hands)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(data_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Example of setting up a data loader and training
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(card_tensors, labels)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
train_loop(classifier, data_loader, criterion, optimizer, num_epochs=100)


In [17]:
# ---- function that outputs the softmax probability for each class >= 1.00%, formatted to .00% and sorted largest to smallest % ----
def get_class_probabilities(model, hand_tensor):
    model.eval()
    with torch.no_grad():
        encoded_cards = encoder(hand_tensor.view(-1, 2))
        encoded_hand = encoded_cards.view(1, 5, -1)
        outputs = model(encoded_hand)
        probabilities = F.softmax(outputs, dim=1).squeeze(0)
        significant_probs = {reverse_poker_hand_index(i): prob.item() for i, prob in enumerate(probabilities)}
        sorted_probs = dict(sorted(significant_probs.items(), key=lambda item: item[1], reverse=True))
        formatted_probs = {k: f"{v*100:.2f}%" for k, v in sorted_probs.items()}
        return formatted_probs

In [26]:
# ---- print tensor hands, expected output, and classifier outputs ----
for i in range(card_tensors.size(0)):
    hand_tensor = card_tensors[i].unsqueeze(0)
    expected_label = labels[i].item()
    hand_type = reverse_poker_hand_index(expected_label)
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    #predicted_label = torch.argmax(outputs, dim=1).item()
    hand = card_strings[i]
    print(f"\nhand: {hand}")
    print(f"correct label confidence: {hand_type} -- {probabilities[hand_type]}")

# ---- test to see if the model can correctly predict each royal flush hand ----
royal_flush_hands = [
    ["A♣", "K♣", "Q♣", "J♣", "X♣"],
    ["A♦", "K♦", "Q♦", "J♦", "X♦"],
    ["A♥", "K♥", "Q♥", "J♥", "X♥"],
    ["A♠", "K♠", "Q♠", "J♠", "X♠"],
]

for hand in royal_flush_hands:
    hand_tensor = card_strings_to_tensor(hand).unsqueeze(0)
    hand_type = "royal flush"
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    print(f"\nhand: {hand}")
    print(f"correct label confidence: {hand_type} -- {probabilities[hand_type]}")



hand: ['A♥', 'K♦', 'Q♣', 'J♠', '__']
correct label confidence: high card -- 92.57%

hand: ['A♥', 'K♥', 'Q♥', 'J♥', 'X♥']
correct label confidence: royal flush -- 94.76%

hand: ['9♠', '8♠', '7♠', '6♠', '5♠']
correct label confidence: straight flush -- 93.84%

hand: ['3♦', '3♣', '3♥', '3♠', '9♥']
correct label confidence: four of a kind -- 96.75%

hand: ['4♥', '4♠', '4♣', '9♦', '9♥']
correct label confidence: full house -- 97.14%

hand: ['2♣', '5♣', '9♣', 'J♣', 'K♣']
correct label confidence: flush -- 96.68%

hand: ['6♥', '5♠', '4♦', '3♣', '2♥']
correct label confidence: straight -- 96.17%

hand: ['7♠', '7♥', '7♣', '2♦', '5♥']
correct label confidence: three of a kind -- 96.17%

hand: ['8♦', '8♣', '4♥', '4♠', 'K♥']
correct label confidence: two pair -- 98.25%

hand: ['J♣', 'J♠', '3♥', '6♦', '9♣']
correct label confidence: one pair -- 97.71%

hand: ['5♥', 'X♠', '7♣', '4♦', '2♥']
correct label confidence: high card -- 96.10%

hand: ['5♥', '5♠', '__', '__', '__']
correct label confidence: 

IndexError: index out of range in self

In [18]:
royal_flush_hands = [
    ["__", "A♣", "K♦", "Q♥", "J?"],
]

for hand in royal_flush_hands:
    hand_tensor = card_strings_to_tensor(hand).unsqueeze(0)
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    print(f"\nhand: {hand}")
    print(f"class probabilities: {probabilities}")


hand: ['__', 'A♣', 'K♦', 'Q♥', 'J?']
class probabilities: {'nothing': '59.13%', 'high card': '15.94%', 'four of a kind': '13.88%', 'royal flush': '3.91%', 'one pair': '3.15%', 'straight': '1.38%', 'three of a kind': '1.22%', 'full house': '0.84%', 'flush': '0.30%', 'straight flush': '0.17%', 'two pair': '0.10%'}
